# Movielens 영화 추천

### 루브릭

1. CSR matrix가 정상적으로 만들어졌다. / 사용자와 아이템 개수를 바탕으로 정확한 사이즈로 만들었다.  
2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다. / 사용자와 아이템 벡터 내적수치가 의미있게 형성되었다.  
3. 비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다. / MF모델이 예측한 유저 선호도 및 아이템간 유사도, 기여도가 의미있게 측정되었다.

In [4]:
import pandas as pd
import os
import numpy as np
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix

## 1) 데이터 준비와 전처리

In [5]:
rating_file_path=os.getenv('HOME') + '/aiffel/workplace/EX14_Recommender_System_IU/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [8]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [9]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [11]:
# 잘 바뀌었는지 확인

ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [17]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.

movie_file_path=os.getenv('HOME') + '/aiffel/workplace/EX14_Recommender_System_IU/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


## MF model 구성

In [18]:
# idx_to_movie
idx_to_movie = {}
for i in range(len(movies)):
    idx_to_movie[movies['movie_id'][i]] = movies['title'][i]

In [19]:
# movie_to_idx
movie_to_idx = {}
for k, v in idx_to_movie.items():
    movie_to_idx[v] = k

In [20]:
# 나의 추천 목록 생성
my_favorite = [2, 10, 247, 316, 345]
my_movielist = pd.DataFrame({'user_id':[6041]*5, 'movie_id':my_favorite, 'counts':[5, 4, 4, 3, 5]})

my_movielist

,user_id,movie_id,counts
0,6041,2,5
1,6041,10,4
2,6041,247,4
3,6041,316,3
4,6041,345,5


In [21]:
if not ratings.isin({'user_id':[6041]})['user_id'].any():
    ratings = ratings.append(my_movielist)

In [22]:
# ratings에 있는 유니크한 유저수
ratings['user_id'].nunique()

6040

In [23]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [24]:
# 가장 인기 있는 영화 30개 (인기순)
movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

In [25]:
# 유저별 몇 개의 영화를 봤는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6040.000000
mean      138.490563
std       156.238108
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [26]:
# csr_data의 출력 차원
num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

print(num_user, num_movie)

6040 3628


In [27]:
# csr_data 생성
csr_data = csr_matrix((ratings['counts'], (ratings['user_id'], ratings['movie_id'])))
csr_data # shape가 일치하지 않아서 ValueError가 발생하였기 때문에 shape parameter를 비워두었다.

<6042x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [28]:
# implicit 라이브러리에서 권장하고 있는 부분
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [30]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [31]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3953x6042 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [32]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [34]:
# 벡터값 확인
my_vector, movie_vector = als_model.user_factors[6041], als_model.item_factors[2]

In [35]:
my_vector

array([ 0.3631876 ,  0.08780039, -0.53182256, -0.3561194 , -0.38550097,
        0.09879929, -0.49542835, -0.3871193 ,  0.5871021 ,  0.621338  ,
        0.98676175,  0.65514416,  0.60065764, -1.139925  ,  0.22890615,
        0.4010421 ,  0.78902006,  0.3225129 , -0.4213978 , -0.1187789 ,
        0.37179735, -0.28343073,  0.3120577 ,  0.42702478, -0.7152035 ,
        0.22388288,  0.35395083,  0.24987434, -0.2030514 ,  0.33377   ,
       -0.6559256 ,  0.29627147,  0.73240304,  0.11363308,  0.24810076,
       -0.00952933, -0.15408888, -0.7639335 , -0.02887493,  0.13741447,
        0.21956949,  0.15626289,  0.05902308,  0.02992974,  0.4066699 ,
        0.32997662, -0.03191189,  0.37691802,  0.01064063, -1.0565331 ,
       -0.46785307, -0.21938956,  0.20366007, -0.3013407 , -0.4957362 ,
        0.46869346,  0.08541214,  0.3609393 ,  0.3305935 , -0.27362832,
       -0.40919632,  0.1697632 ,  0.3525702 ,  0.03246704, -0.71753126,
       -0.48928708,  0.20337127,  0.21568765, -0.04424765, -0.25

In [36]:
movie_vector

array([ 1.63611881e-02, -1.51322801e-02, -2.97776307e-03, -8.90906248e-03,
        7.89926760e-03,  1.71154495e-02, -1.75068434e-02,  9.87213384e-03,
        3.13047692e-02,  1.58092976e-02,  3.29915211e-02,  4.27366868e-02,
        1.83154438e-02, -2.50831265e-02,  3.45435832e-03,  1.95983215e-03,
        3.03186793e-02,  4.48477129e-03,  1.21044051e-02, -5.82448626e-03,
        1.79809667e-02,  3.08140717e-03,  1.97172686e-02, -1.34153506e-02,
       -1.18432473e-02,  9.09868255e-03,  1.45438239e-02,  1.92244444e-02,
       -3.90217733e-03,  7.82644749e-03,  1.41810188e-02, -9.50320344e-03,
        3.09442114e-02, -1.47028137e-02, -9.71434638e-03, -2.01952201e-03,
        4.98214876e-03, -1.66685954e-02,  7.56839849e-03,  2.30553355e-02,
        1.43350121e-02,  8.66211951e-03,  1.73308060e-03,  8.06595571e-03,
        9.38079975e-05,  1.56875029e-02,  6.27204357e-03,  2.23174896e-02,
        4.50907648e-03, -2.26749312e-02, -2.14356743e-02,  1.16076379e-03,
        8.87546781e-03,  

In [37]:
# 내적
np.dot(my_vector, movie_vector)

0.3242845

In [45]:
# 비슷한 영화 찾기
favorite_movie_id = 10
similar_movie = als_model.similar_items(favorite_movie_id, N=15)
similar_movie

[(10, 1.0000002),
 (1722, 0.78572017),
 (3082, 0.6273495),
 (349, 0.6081375),
 (165, 0.57621413),
 (2006, 0.56897694),
 (2273, 0.56718653),
 (733, 0.55067194),
 (112, 0.51293075),
 (2990, 0.50671434),
 (1377, 0.50013983),
 (1370, 0.49602485),
 (3256, 0.48900762),
 (380, 0.48849198),
 (1552, 0.47873)]

In [46]:
[idx_to_movie[i[0]] for i in similar_movie]

['GoldenEye (1995)',
 'Tomorrow Never Dies (1997)',
 'World Is Not Enough, The (1999)',
 'Clear and Present Danger (1994)',
 'Die Hard: With a Vengeance (1995)',
 'Mask of Zorro, The (1998)',
 'Rush Hour (1998)',
 'Rock, The (1996)',
 'Rumble in the Bronx (1995)',
 'Licence to Kill (1989)',
 'Batman Returns (1992)',
 'Die Hard 2 (1990)',
 'Patriot Games (1992)',
 'True Lies (1994)',
 'Con Air (1997)']

In [47]:
# 비슷한 영화를 찾아주는 함수

def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [50]:
get_similar_movie('Die Hard 2 (1990)')

['Die Hard 2 (1990)',
 'Die Hard: With a Vengeance (1995)',
 'Enemy of the State (1998)',
 'Patriot Games (1992)',
 'Clear and Present Danger (1994)',
 'Air Force One (1997)',
 'Negotiator, The (1998)',
 'Speed (1994)',
 'Rock, The (1996)',
 'Hunt for Red October, The (1990)']

In [51]:
# 비슷한 영화를 찾아주는 함수

def get_similar_movie_id(movie_id: str):
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [53]:
# Toy Story와 비슷한 영화 추천
get_similar_movie_id(10)

['GoldenEye (1995)',
 'Tomorrow Never Dies (1997)',
 'World Is Not Enough, The (1999)',
 'Clear and Present Danger (1994)',
 'Die Hard: With a Vengeance (1995)',
 'Mask of Zorro, The (1998)',
 'Rush Hour (1998)',
 'Rock, The (1996)',
 'Rumble in the Bronx (1995)',
 'Licence to Kill (1989)']

## 회고

- 역시 이번 추천시스템을 진행하면서 느낀 것 중에 하나는 '도메인에 대한 이해'와 '데이터에 대한 이해'가 절실하게 필요하다는 것이었다.  
- 데이터 사이언티스트를 하건, 인공지능 엔지니어를 하건간에 전체 분야에 대한 기본적인 통찰력과 도메인에 대한 어느정도의 전문지식이 필요다다는 것을 다시한번 깨닳은 프로젝트였다.  
- 아무생각 없이 재미있게 보았던 그리고 어떻게 내가 좋아 하는 스타일의 영화를 저렇게도 콕콕 집어서 추천해줄까 궁금했던 넷플렉스의 추천시스템에 대해서 제대로 이해 할 수 있는 시간이었다.
- 오래전에 재미 있게 보았던 부루스윌리스 주연의 Die Hard 2 (1990)를 선택했고 결과적으로 전반적으로 비슷한 영화를 찾아주었고, 제대로 된 추천을 진행한 것 같이 보인다. 